In [ ]:
from vacances_scolaires_france import SchoolHolidayDates
from jours_feries_france import JoursFeries
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn


def data_constructor(data):
    df_original = data
    
    # open the data file
    df_external = pd.read_csv("external_data.csv")

    # create the quarantine columns
    df_external["quarantine1"] = np.where(
    (df_external["date"] <= "2020-12-15") & (df_external["date"] >= "2020-10-30"), "True", "False")
    df_external["quarantine2"] = np.where(
    (df_external["date"] <= "2021-05-03") & (df_external["date"] >= "2021-04-03"), "True", "False")


    #creating two new variables, holiday and bank_holiday to represent whether a given date is a school holiday or a bank holiday
    hol = SchoolHolidayDates()

    df_external['holiday'] = df_external.apply(
    lambda row: hol.is_holiday_for_zone(datetime.strptime(row["date"], '%Y-%m-%d %H:%M:%S').date(), 'C'), axis=1)

    df_external['bank_holiday'] = df_external.apply(
    lambda row: JoursFeries.is_bank_holiday(datetime.strptime(row["date"], '%Y-%m-%d %H:%M:%S').date(), zone="Métropole"), axis=1)

    # Obtenez le nombre total de lignes dans le DataFrame externe
    n_rows = df_external.shape[0]

    # Calculez la proportion de valeurs manquantes pour chaque colonne
    proportion_missing_values = df_external.isnull().sum() / n_rows

    # Sélectionnez les colonnes à supprimer (avec un seuil de 10% de valeurs manquantes)
    columns_to_drop = proportion_missing_values[proportion_missing_values >= 0.1].index

    # Supprimez les colonnes sélectionnées du DataFrame externe
    df_external = df_external.drop(columns=columns_to_drop)

    col_to_delete_2 = ["numer_sta", "cod_tend","pmer","nbas", "rr1" ,"rr3","rr12","rr6","rr24", "nbas", "ht_neige", "rafper","ff", "per", "w2", "td"]

    # Supprimez les colonnes sélectionnées du DataFrame externe
    df_external = df_external.drop(columns=col_to_delete_2)
    
    # Remplace les valeurs manquantes par la médiane
    df_external = df_external.fillna(df_external.median(numeric_only=True))

    df_external['date'] = pd.to_datetime(df_external['date'], errors='coerce')

    df_merge = pd.merge_asof(df_original.sort_values("date"), df_external.sort_values("date"), on="date")
    
    return df_merge
    
    